In [4]:
import data_cleaning
import json
import pandas as pd
import sys
import re
!{sys.executable} -m pip install langdetect

In [5]:
wikipedia_parsed = "../data/json_files/grawitas_output/wikipedia_parsed.json"
wikidata_parsed = "../data/json_files/grawitas_output/wikidata_parsed.json"
meta_parsed = "../data/json_files/grawitas_output/meta_parsed.json"
with open(wikipedia_parsed) as f:
        wikipedia_list_of_dicts = json.load(f)  

with open(wikidata_parsed) as f:
        wikidata_list_of_dicts = json.load(f) 
        
with open(meta_parsed) as f:
        meta_list_of_dicts = json.load(f)  

In [6]:
wiki_comments, wiki_rfc = data_cleaning.get_RFC_Comment_Table(wikipedia_list_of_dicts, wikidata_list_of_dicts, meta_list_of_dicts)

In [7]:
comment_df = pd.json_normalize(wiki_comments, "page_text", ["page_title","page_id"])

In [8]:
rfc_df = pd.json_normalize(wiki_rfc)

In [9]:
data_cleaning.templatesToReadableText(comment_df)

100%|██████████| 102474/102474 [00:49<00:00, 2088.53it/s]


In [10]:
from langdetect import detect
import re
def detectLanguage(text):
    try:
        return detect(text)
    except:
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        url = re.findall(regex, text)
        if url:
            return "en"

In [11]:
# define regular expression pattern for matching non-word characters
non_word_pattern = re.compile(r'^\W*$')

non_alpha_pattern = re.compile(r'^[^a-zA-Z]+$')

# Define regex pattern
pattern = '\s*15px\|[a-zA-Z]+=\s*\|[a-zA-Z]+=[a-zA-Z]+\s*\|'

In [12]:
from tqdm import tqdm
tqdm.pandas()

comment_df['text'] = comment_df['text'].str.replace(pattern, '')

# filter dataframe to remove rows that contain only non-alpha characters
comment_df = comment_df[~comment_df['text'].str.contains(non_alpha_pattern)]

# filter dataframe to remove rows that contain only non-word characters
comment_df = comment_df[~comment_df['text'].str.contains(non_word_pattern)]

# Add a new column to the DataFrame indicating the language of the text
comment_df['language'] = comment_df['text'].progress_apply(detectLanguage)

comment_df = comment_df[comment_df['language'] == 'en']

/tmp/ipykernel_9097/4090611691.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  comment_df['text'] = comment_df['text'].str.replace(pattern, '')
100%|██████████| 101449/101449 [12:18<00:00, 137.36it/s]


In [13]:
gfg_csv_data = rfc_df.to_csv('../data/rfc.csv', index = False)
print('\nCSV String:\n', gfg_csv_data) 


CSV String:
 None


In [14]:
#comment_df.loc[60187,'date'] = '2022-03-22T23:36:00Z'
gfg_csv_data = comment_df.to_csv('../data/rfc_comments.csv', index = False)
print('\nCSV String:\n', gfg_csv_data) 


CSV String:
 None


In [15]:
def getcleanDataFrame(df):
    #remove IP-Adresses from userArray
    df = df[~df['user'].str.contains(re.compile(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b'))]
    return list(df["user"].unique())

user_list = getcleanDataFrame(comment_df)

In [16]:
print(len(user_list))

9363


In [17]:
def getUserInfoToJSON(userArray, output):
    """
    Takes list of users outputs list of JSON objects containing wiki projects, user rights, edit count, registration and first comment dates
    """ 
    with multiprocessing.Pool(processes=8) as pool:
        results = list(tqdm.tqdm(pool.imap(worker, userArray)))
        with open(output, 'w') as file:
            json.dump(list(results), file)
    
    
def worker(user):
    try:
        userDic = getUserInfoAcrossAllReplicaDatabases(user)
        return userDic
    except:
        print(f"failed to get userDic, with {user}")

In [20]:
from userinformation import getUserInfoToJSON
getUserInfoToJSON(user_list, "../json_files/user_info/users.json")

0it [00:00, ?it/s]

failed to get userDic, with Buffsfailed to get userDic, with EllenCTfailed to get userDic, with Cinderella157failed to get userDic, with Schazjmdfailed to get userDic, with Redrose64failed to get userDic, with DaRonPaynefailed to get userDic, with BC1278








1it [00:00,  2.37it/s]

failed to get userDic, with Coretheapplefailed to get userDic, with Norenfailed to get userDic, with Barkeep49failed to get userDic, with Cthomas3failed to get userDic, with Cynistrategusfailed to get userDic, with Eggishornfailed to get userDic, with Gråbergs Gråa Sång






failed to get userDic, with Debresserfailed to get userDic, with Ermenrichfailed to get userDic, with JzGfailed to get userDic, with GPinkerton




failed to get userDic, with Rosguillfailed to get userDic, with Slaterstevenfailed to get userDic, with Jenhawk777

failed to get userDic, with Smeat75failed to get userDic, with P Aculeiusfailed to get userDic, with Aquillion


failed to get userDic, with Buidhefailed to get userDic, with Ar2332
failed to get userDic, with Llywrch
failed to get userDic, with FyzixFighter

failed to get userDic, with Vevervefailed to get userDic, with Robert McClenon

failed to get userDic, with Pyrite Profailed to get userDic, with Pincretefailed to get userDic, with Marcocapelle
fail

45it [00:00, 112.28it/s]

...

failed to get userDic, with Novem Linguae
failed to get userDic, with BD2412
failed to get userDic, with Joe Roe
failed to get userDic, with Kudpung
failed to get userDic, with DoubleGrazing
failed to get userDic, with Gusfriend
failed to get userDic, with Hellknowz
failed to get userDic, with Thingofme
failed to get userDic, with No such user
failed to get userDic, with Stiflefailed to get userDic, with Mhawk10

failed to get userDic, with Espresso Addict
failed to get userDic, with Jontesta
failed to get userDic, with Herostratus
failed to get userDic, with The Blade of the Northern Lights
failed to get userDic, with JPxG
failed to get userDic, with Clovermoss

490it [00:02, 137.45it/s]


failed to get userDic, with Reywas92
failed to get userDic, with Visviva
failed to get userDic, with Lugnuts
failed to get userDic, with Chris troutman
failed to get userDic, with Phil Bridger
failed to get userDic, with Krantler
failed to get userDic, with Extraordinary Writfailed to get userDic, with SnowFire

failed to get userDic, with Scope creep


500it [00:05, 83.37it/s] 


KeyboardInterrupt: 

In [15]:
comment_df = pd.read_csv('rfc_comments.csv')

In [17]:
# group the dataframe by class and sample 65 rows from each group
df_labelling = comment_df.groupby('project').apply(lambda x: x.sample(65)).reset_index(drop=True)

In [18]:
# Define a function to generate URLs based on page IDs
def get_wikipedia_url(row):
    page_id = row['page_id']
    project = row['project']
    return f'https://{project}.org/wiki?curid={page_id}'

# Apply the function to the page_id column and assign the result to a new column
df_labelling['page_url'] = df_labelling.apply(get_wikipedia_url, axis = 1)

In [19]:
# Create a list of strings to add as new columns
new_cols = [ 'disrespect','respect','explanation','causal_reasoning','narrative', 'question', 'response', 'advocacy', 'public_interest','counterarguments', 'constructive_proposal']

# Add the new columns to the DataFrame
for col in new_cols:
    df_labelling[col] = ''

In [20]:
df_labelling = df_labelling.drop(["date","section","page_id", "rfc_id", "parent_id", "language"], axis=1)

In [21]:
# saving the DataFrame as a CSV file
gfg_csv_data = df_labelling.to_csv('label_rfc_statements.csv', index = False)
print('\nCSV String:\n', gfg_csv_data) 


CSV String:
 None
